In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

import joblib
import random
import sys

from mip import Model as ModelMip, xsum, minimize, maximize, \
INTEGER, BINARY, CONTINUOUS, CutType, OptimizationStatus

import data_generator
from model import StrongStandardNet, StrongVariationalNet
from train import TrainDecoupled, TrainCombined

from sklearn.preprocessing import StandardScaler

import torch.multiprocessing as mp

In [2]:
import cvar_minimization_utils as cmu

In [3]:
is_cuda = False
dev = torch.device('cpu')  
if torch.cuda.is_available():
    is_cuda = True
    dev = torch.device('cuda') 

In [4]:
method_name = 'bnn'
method_learning = 'combined'
aleat_bool = 1

N_SAMPLES = 32
BATCH_SIZE_LOADER = 64

EPOCHS = 100

cpu_count = mp.cpu_count()

seed_number = 0

lr = 0.0005

In [5]:
np.random.seed(seed_number)
torch.manual_seed(seed_number)
random.seed(seed_number)

assert (method_name in ['ann','bnn','gp'])

if method_name in ['ann','bnn']:
    assert (method_learning in ['decoupled','combined'])
    assert (aleat_bool in [True, False])
    assert (N_SAMPLES>=1 and N_SAMPLES<9999)
    #assert (M_SAMPLES>=1 and M_SAMPLES<9999)

bnn = False 
if method_name == 'bnn':
    bnn = True   
    K = 1 # Hyperparameter for the training in ELBO loss
    PLV = 1 # Prior in ELBO loss   

model_name = method_name + '_constrained_'
for i in range(2, len(sys.argv)):
    model_name += '_'+sys.argv[i]
model_name += '_'+ str(seed_number)

In [6]:
def gen_intermidiate(num, n_assets, x1, x2, x3):      
        factor = num * 2/(n_assets)
        return x1**factor + x2**factor + x3**factor
    
def gen_data(N, n_assets, nl, seed_number=42):
    np.random.seed(seed_number)
    x1 = np.random.normal(1, 1, size = N).clip(0)
    x2 = np.random.normal(1, 1, size = N).clip(0)
    x3 = np.random.normal(1, 1, size = N).clip(0)
    X = np.vstack((x1, x2, x3)).T

    Y = np.zeros((N, n_assets))
    for i in range(1, n_assets + 1):
        interm = gen_intermidiate(i, n_assets, x1, x2, x3)
        Y[:,i-1] = (np.sin(interm) - np.sin(interm).mean()) \
        + np.sin(interm).std()*(-0.2 + np.random.random())
        
        ns = Y[:,0].shape[0]//2
        
        Y[:,i-1] = Y[:,i-1] + nl*Y[:,i-1]*x1*(np.random.beta(5, 2, size = Y[:,0].shape) - 0.2)

    return X, Y

def gen_cond_dist(N, n_assets, n_samples, nl, seed_number=420):
    np.random.seed(seed_number)
    Y_dist = np.zeros((n_samples, N, n_assets))
    for i in range(0, n_samples):
        Y_dist[i, :, :] = gen_data(N, n_assets, nl, seed_number=np.random.randint(0,999999))[1]
    return Y_dist

In [7]:
class CVaROP():
    def __init__(self, beta, n_assets, min_return, Y_train):
        self.beta = beta
        self.n_assets = n_assets
        self.R = min_return 
        self.uy = Y_train.mean(axis=0)
        
        self.zstar = np.zeros_like(Y_train)
        self.alphastar = np.zeros_like(Y_train[:,0])
        self.cvarstar = np.zeros_like(Y_train[:,0])
                        
        #for i in range(0, Y_train.shape[0]):
        #    self.zstar[i,:], _ , self.alphastar[i], self.cvarstar[i] = \
        #    self.minimize_cvar(np.expand_dims(Y_train, 0)[:,i,:])
      
    def minimize_cvar(self, y):
    
        n_samples = y.shape[0]
        n_assets = y.shape[1]
        
        assert self.n_assets == n_assets
        
        
        m = ModelMip("cvar")
        m.verbose = 0
        z = ([m.add_var(var_type=CONTINUOUS, name=f'z_{i}') for i in range(0, n_assets)])
        u = ([m.add_var(var_type=CONTINUOUS, name=f'u_{i}') for i in range(0, n_samples)])
        alpha = m.add_var(var_type=CONTINUOUS, name='alpha')

        m.objective = minimize(alpha + (1/(1 - self.beta))*(1/n_samples) \
                               *xsum(u[i] for i in range(0, n_samples)))

        for i in range(0, n_assets):
            m += z[i] >= 0

        for i in range(0, n_samples):
            m += u[i] >= 0

        for i in range(0, n_samples):
            m += xsum(z[j]*y[i][j] for j in range(0, n_assets)) + alpha + u[i] >= 0

        for i in range(0, n_assets):
            m += xsum(-z[i]*self.uy[i] for i in range(0, n_assets)) <= -self.R

        m.optimize()
        f_opt = m.objective_value
        argmins = []
        for v in m.vars:
            argmins.append(v.x)
           
        zstar = argmins[:n_assets]
        ustar = argmins[n_assets:n_assets+n_samples]
        alphastar = argmins[n_assets+n_samples]         

        return zstar, ustar, alphastar, f_opt
    
    
    def minimize_cvar_dataset(self, Y_dist):
        zstar = np.zeros_like(Y_dist[0])
        for i in range(0, Y_dist.shape[1]):
            zstar[i,:] , _, _, _ = self.minimize_cvar(Y_dist[:,i,:])       
        return zstar
    
    def minimize_cvar_given_z(self, y, z):
    
        n_samples = y.shape[0]
        n_assets = y.shape[1]
        
        assert self.n_assets == n_assets
        
        
        m = ModelMip("cvar")
        m.verbose = 0

        u = ([m.add_var(var_type=CONTINUOUS, name=f'u_{i}') for i in range(0, n_samples)])
        alpha = m.add_var(var_type=CONTINUOUS, name='alpha')

        m.objective = minimize(alpha + (1/(1 - self.beta))*(1/n_samples) \
                               *xsum(u[i] for i in range(0, n_samples)))

        for i in range(0, n_samples):
            m += u[i] >= 0

        for i in range(0, n_samples):
            m += xsum(z[j]*y[i][j] for j in range(0, n_assets)) + alpha + u[i] >= 0

        m.optimize()
        f_opt = m.objective_value
        argmins = []
        for v in m.vars:
            argmins.append(v.x)
           
        ustar = argmins[:n_samples]
        alphastar = argmins[n_samples]    

        return ustar, alphastar, f_opt
    
    
    def minimize_cvar_given_z_dataset(self, Y_dist, z):
        alphastar = np.zeros_like(Y_dist[0,:,0])
        for i in range(0, Y_dist.shape[1]):
            _, alphastar[i] , _ = self.minimize_cvar_given_z(Y_dist[:,i,:], z[i,:])
        return alphastar
        
    def calc_cvar_dataset(self, Y_dist, zstar_pred, alpha):
        cvar_pred = alpha + (1/(1-beta))*(np.maximum(-((Y_dist*zstar_pred).sum(2)) \
                                                              - alpha, 0)).mean(0)
        return cvar_pred
        
    
    def calc_f_dataset(self, Y_dist_pred, Y_dist):
        zstar_pred = self.minimize_cvar_dataset(self, Y_dist_pred)
        alphastar_pred = minimize_cvar_given_z_dataset(self, Y_dist, zstar_pred)
        cvar_pred = self.calc_cvar_dataset(Y_dist, zstar_pred)
        return cvar_pred
    
    
    def cost_fn(self, Y_dist_pred, Y_dist):
        f = self.calc_f_dataset(Y_dist_pred, Y_dist)
        f_total = torch.mean(f)
        return f_total
        
    #def end_loss(self, Y_dist_pred, Y_dist):
    #    y_pred = y_pred.unsqueeze(0)
    #    f_total = self.cost_fn(y_pred, y)
    #    return f_total
    
    def end_loss_dist(self, Y_dist_pred, Y_dist):
        f_total = self.cost_fn(Y_dist_pred, Y_dist)
        return f_total

In [8]:
N_train = 1000
N_val = 600
N_test = 600

n_assets = 15

n_samples_orig = 200

nl = 0.1

X, Y_original = gen_data(N_train, n_assets, nl, seed_number)
X_val, Y_val_original = gen_data(N_val, n_assets, nl, seed_number + 80)
X_test, Y_test_original = gen_data(N_test, n_assets, nl, seed_number + 160)

Y_dist = gen_cond_dist(N_train, n_assets, n_samples_orig, nl=nl, seed_number=seed_number)
Y_val_dist = gen_cond_dist(N_val, n_assets, n_samples_orig, nl=nl, seed_number=seed_number + 100)
Y_test_dist = gen_cond_dist(N_test, n_assets, n_samples_orig, nl=nl, seed_number=seed_number + 200)

In [9]:
# Output normalization
scaler = StandardScaler()
scaler.fit(Y_original)
tmean = torch.tensor(scaler.mean_).to(dev)
tstd = torch.tensor(scaler.scale_).to(dev)
joblib.dump(scaler, 'scaler_cvar.gz')

# Function to denormalize the data
def inverse_transform(yy):
    return yy*tstd + tmean

Y = scaler.transform(Y_original).copy()
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)
data_train = data_generator.ArtificialDataset(X, Y)
training_loader = torch.utils.data.DataLoader(
    data_train, batch_size=BATCH_SIZE_LOADER,
    shuffle=True, num_workers=cpu_count)
Y_dist = torch.tensor(Y_dist, dtype=torch.float32)

Y_val = scaler.transform(Y_val_original).copy()
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_val_original = torch.tensor(Y_val_original, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.float32)
data_valid = data_generator.ArtificialDataset(X_val, Y_val)
validation_loader = torch.utils.data.DataLoader(
    data_valid, batch_size=BATCH_SIZE_LOADER,
    shuffle=False, num_workers=cpu_count)

X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test_original = torch.tensor(
    Y_test_original, dtype=torch.float32)
data_test = data_generator.ArtificialDataset(
    X_test, Y_test_original)
test_loader = torch.utils.data.DataLoader(
data_test, batch_size=16,
shuffle=False, num_workers=cpu_count)

In [10]:
n_samples = Y.shape[0]
n_assets = Y.shape[1]
beta = 0.80
min_return = 100

In [23]:
method_name = 'bnn'
method_learning = 'combined'

bnn = False
aleat_bool = False
if method_name == 'bnn':
    bnn = True
    aleat_bool = True
N_SAMPLES = 16
K = 1

In [24]:
lr = 0.0005

In [25]:
if method_name == 'bnn':
    h = StrongVariationalNet(
    n_samples=N_SAMPLES,
    input_size=X.shape[1], 
    output_size=Y.shape[1], 
    plv=PLV, 
    dev=dev
    ).to(dev)

    
#ANN Baseline model
elif method_name == 'ann':
    h = StrongStandardNet(X.shape[1], Y.shape[1]).to(dev)
    K = 0 # There is no K in ANN
    N_SAMPLES = 1
    
opt_h = torch.optim.Adam(h.parameters(), lr=lr)
mse_loss = nn.MSELoss(reduction='none')


#opt_cvar_gZ = cmu.CVaROPgZ(1, n_assets, beta, min_return, torch.tensor(Y_original), dev)
op = cmu.RiskPortOP(N_SAMPLES, n_assets, min_return, torch.tensor(Y_original), dev)

# Decoupled learning approach
if method_learning == 'decoupled':
    train_NN = TrainDecoupled(
                    bnn = bnn,
                    model=h,
                    opt=opt_h,
                    loss_data=mse_loss,
                    K=K,
                    aleat_bool=aleat_bool,
                    training_loader=training_loader,
                    validation_loader=validation_loader,
                    dev=dev
                )

# Combined learning approach (end-to-end loss)
elif method_learning == 'combined':
    train_NN = TrainCombined(
                    bnn = bnn,
                    model=h,
                    opt=opt_h,
                    K=K,
                    aleat_bool=aleat_bool,
                    training_loader=training_loader,
                    scaler=scaler,
                    validation_loader=validation_loader,
                    OP=op,
                    dev=dev
                )

In [26]:
model_used = train_NN.train(EPOCHS=200, pre_train=-1)

  0%|▎                                                                   | 1/200 [00:03<11:13,  3.39s/it]

------------------EPOCH 1------------------
END LOSS 	 train 27.249 valid 26.922 
 KL LOSS 	 train 4.321 valid 4.319 
 TOTAL LOSS 	 train 31.569 valid 31.241 



  1%|▋                                                                   | 2/200 [00:06<10:54,  3.30s/it]

------------------EPOCH 2------------------
END LOSS 	 train 26.711 valid 26.43 
 KL LOSS 	 train 4.318 valid 4.316 
 TOTAL LOSS 	 train 31.029 valid 30.746 



  2%|█                                                                   | 3/200 [00:09<10:50,  3.30s/it]

...

------------------EPOCH 198------------------
END LOSS 	 train 2.551 valid 1.522 
 KL LOSS 	 train 4.134 valid 4.133 
 TOTAL LOSS 	 train 6.685 valid 5.655 



100%|█████████████████████████████████████████████████████████████████▋| 199/200 [13:12<00:04,  4.06s/it]

------------------EPOCH 199------------------
END LOSS 	 train 2.506 valid 1.173 
 KL LOSS 	 train 4.133 valid 4.133 
 TOTAL LOSS 	 train 6.639 valid 5.307 



100%|██████████████████████████████████████████████████████████████████| 200/200 [13:16<00:00,  3.98s/it]

------------------EPOCH 200------------------
END LOSS 	 train 2.575 valid 1.021 
 KL LOSS 	 train 4.133 valid 4.133 
 TOTAL LOSS 	 train 6.708 valid 5.154 



In [27]:
M_opt = 128
model_used.update_n_samples(n_samples=M_opt)
Y_pred = model_used.forward_dist(X_test, aleat_bool)
Y_pred_original = inverse_transform(Y_pred)

In [28]:
op = cmu.RiskPortOP(M_opt, n_assets, min_return, torch.tensor(Y_original), dev)
op.end_loss_dist(Y_pred_original, Y_test_original)

tensor(3.2776, dtype=torch.float64, grad_fn=<MeanBackward0>)